In [2]:
from sunflower.config import *
from sunflower.core import *

ModuleNotFoundError: No module named 'config'

In [4]:
Raw(user['stephen'],payslip['files']).read_files()

   

{'Cohort 1 - Payslips 01Jul16 to 30Jun17 - excl BNT & Merch.xlsx':         ID Number Date of Birth  Hire Date  Level 1 Code  Level 3 Code  \
 0           16474    1981-12-18 2010-05-10            19            94   
 1           16474    1981-12-18 2010-05-10            19            94   
 2           16474    1981-12-18 2010-05-10            19            94   
 3           16474    1981-12-18 2010-05-10            19            94   
 4           16474    1981-12-18 2010-05-10            19            94   
 ...           ...           ...        ...           ...           ...   
 171256     991127    1995-06-14 2016-11-22            19            94   
 171257     991127    1995-06-14 2016-11-22            19            94   
 171258     991127    1995-06-14 2016-11-22            19            94   
 171259     991127    1995-06-14 2016-11-22            19            94   
 171260     991127    1995-06-14 2016-11-22            19            94   
 
        Account Code   Account C

In [ ]:
f=['sdsdsdsd corhort','mydir']

a=[ i for i in f if any( x in i for x in ['corhort','sdsdsdsd'])]

reporting_df.head(100).to_csv ('test.csv', index = False, header=True)

In [ ]:
a

In [ ]:
reporting_df

In [ ]:
def perform(fun, *args):
    a=fun(*args)
    return a

def action1(x):
    x=x+1
    return x

def action2(x):
    x=x+1
    return x


perform(action2, 5)
